In [1]:
%pip install --quiet --upgrade -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Dependencies and Tools

In [2]:
from langchain_core.messages import HumanMessage

In [3]:
# Define here maximum web search results for the web scraper

MAX_WEB_SEARCH_RESULTS = 5

In [4]:
# Agent tools

tools = []

In [5]:
import os

os.environ["LANGSMITH_TRACING"] = "true"

In [6]:
#Env File

from dotenv import load_dotenv

load_dotenv("trustme.env");

In [7]:
from langchain.prompts import SystemMessagePromptTemplate

system_prompt = SystemMessagePromptTemplate.from_template(
    '''
    Quando l'utente chiede di valutare la fiducia o l'affidabilità di una persona specifica 
    (es. 'Quanto posso fidarmi di Mario Rossi?'), usa il tool 'search' passando il nome della persona.
    '''
)

In [8]:
# LLM 

from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

In [9]:
#Embeddings Model

from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_EMBEDDINGS_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("AZURE_OPENAI_EMBEDDINGS_API_VERSION"),
)

In [10]:
# Vector DB

from langchain_community.vectorstores.azuresearch import AzureSearch

vector_store = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
    azure_search_key=os.getenv("AZURE_SEARCH_API_KEY"),
    index_name=os.getenv("AZURE_SEARCH_INDEX_NAME"),
    embedding_function=embeddings.embed_query,
)

In [ ]:
# Web Search Tool

from langchain_tavily import TavilySearch

search = TavilySearch(
    max_results=MAX_WEB_SEARCH_RESULTS,
    description="Usa questo tool per cercare informazioni su una persona specifica quando l'utente chiede di valutare la fiducia o l'affidabilità di qualcuno, fornendo il nome."
)

tools.append(search)

/tmp/ipykernel_305800/3213391993.py:5: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(


### Implementation

In [13]:
from langgraph.prebuilt import create_react_agent

personal_agent = create_react_agent(llm, tools)

In [ ]:
while True:

    user_message = input("Query (exit per uscire)")

    if user_message == "exit" or user_message == "esci" or not user_message:
        break

    print(f"User message: {user_message}")

    response = personal_agent.invoke(
        {"messages": [HumanMessage(content=user_message)]
    })

    print(f"Agent message: {response['messages']}")
    
    if "tool_calls" in response and len(response["tool_calls"]) > 0:

        for tool_call in response["tool_calls"]:
            print(f"Chiamata metodo {tool_call['name']}")

User message: Valuta la fiducia di mario rossi.
Agent message: [HumanMessage(content='Valuta la fiducia di mario rossi.', additional_kwargs={}, response_metadata={}, id='72625af2-24d0-4eaf-ae0d-5bf7192cf529'), AIMessage(content='“Mario Rossi” è un nome molto comune (spesso usato come esempio), perciò, senza ulteriori dettagli (ad esempio: professione, ambito di attività, luogo di residenza, eventuali fonti o riferimenti specifici), non posso formulare una valutazione attendibile sulla sua affidabilità.  \nMi puoi fornire qualche informazione in più su quale “Mario Rossi” intendi?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 364, 'prompt_tokens': 94, 'total_tokens': 458, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o4-mini-2025-04-16', 'system_fingerprint': None,